In [1]:
import sys
import pickle
import numpy as np
import progress
from progress.bar import IncrementalBar
from keras.models import load_model
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.config.set_visible_devices([], 'GPU')

2023-12-11 14:08:50.194828: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 14:08:51.072718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 14:08:52.324992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-11 14:08:54.524994: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
thresholds = [0.20, 0.25, 0.30, 0.35, 0.40, 0.45,0.50, 0.55, 0.60]

# Loading the test dataset and the model 
f = open("X_test.pkl", "rb")
X_test = pickle.load(f)
f.close()

model = load_model("model.h5")

# Compute the predictions for every threshold
predictions = []
bar = IncrementalBar('Processing', max=9)
for threshold in thresholds:
    predictions_threshold = np.zeros((len(X_test), 11))
    for (key,val) in X_test.items():
        # Predicting on the subset
        # Reshape the test example to essentially form multiple sliding windows of fixed length.
        val = np.reshape(val, (val.shape[0], val.shape[1], val.shape[2], 1))
        #Predictions seem to crash on Tensorflow GPU. Explicitly setting to run predictions on CPU.
        with tf.device('/cpu:0'):
            prediction = model.predict(val)
        
        # Aggregate the results from the sliding window. Sum and Normalize the sigmoid outputs.
        prediction = np.sum(prediction, axis = 0)
        m = np.max(prediction)
        prediction /= m
        prediction = prediction - threshold
        prediction[prediction > 0] = 1
        prediction[prediction <= 0] = 0
        predictions_threshold[key] = prediction
    predictions.append(predictions_threshold)
    bar.next()
bar.finish()

# Saving the predictions 
pickle.dump(predictions, open("predictions", "wb"))
print("Predictions completed.")

1/1 [==============================] - 0s 47ms/step
